In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
#the below three bring in the parts to test the model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [2]:
from sklearn.model_selection import GridSearchCV

In [3]:
pd.set_option('display.max_columns', 300)

In [4]:
df = pd.read_csv('dummies_log.csv')

In [6]:
df.head(6)
#need to remove the old index

,manufacturer,condition,fuel,title_status,transmission,type,state_name,weather,no_cyls,age,vin_yn,odo_yn,log_price,log_age,Alfa Romeo,Aston Martin,Audi,BMW,Buick,Cadillac,Chevrolet,Chrysler,Dodge,Ferrari,Fiat,Ford,GMC,Honda,Hyundai,Infiniti,Jaguar,Jeep,Kia,Land Rover,Lexus,Lincoln,Mazda,Mercedes,Mercury,Mini,Mitsubishi,Nissan,Pontiac,Porsche,Ram,Saturn,Subaru,Toyota,Volkswagen,Volvo,manual,other,lien,missing,rebuilt,gas,hybrid/electric,bus,convertible,coupe,hatchback,mini-van,offroad,other.1,pickup,sedan,truck,van,wagon,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Florida,Georgia,Hawaii,Idaho,Illinois,Indiana,Iowa,Kansas,Kentucky,Louisiana,Maine,Maryland,Massachusetts,Michigan,Minnesota,Mississippi,Missouri,Montana,Nebraska,Nevada,New Hampshire,New Jersey,New Mexico,New York,North Carolina,North Dakota,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
0,Dodge,good,gas,clean,automatic,coupe,South Dakota,38.0,6,8.5,False,True,9.384294,2.140066,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,Ford,good,gas,clean,automatic,sedan,Iowa,47.0,6,14.5,True,True,7.937375,2.674149,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Jeep,like new,gas,rebuilt,automatic,SUV,South Dakota,38.0,4,3.5,False,True,9.433484,1.252763,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,Ford,fair,gas,clean,automatic,sedan,Texas,67.0,4,14.5,False,False,7.130899,2.674149,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,Cadillac,excellent,gas,clean,automatic,sedan,Texas,67.0,6,9.5,False,True,8.999619,2.251292,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,Land Rover,excellent,gas,clean,automatic,SUV,Nevada,46.0,8,14.5,False,True,9.190138,2.674149,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [7]:
x = df.drop(['condition', 'manufacturer', 'state_name', 'type', 'transmission', 'title_status', 'fuel', 'age'], axis=1)
y = df['condition']

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 0, test_size = 0.3)

In [9]:
knn_classifier = KNeighborsClassifier(n_neighbors = 50, p=2, metric= 'euclidean')

In [10]:
knn_classifier.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=None, n_neighbors=50, p=2,
           weights='uniform')

In [ ]:
y_pred = knn_classifier.predict(x_test)

In [ ]:
f1_score(y_test, y_pred, average = 'macro')

In [ ]:
y_pred

In [ ]:
cm = confusion_matrix(y_test, y_pred)

In [ ]:
cm

In [ ]:
accuracy_score(y_test, y_pred)

## Grid Search

In [ ]:
param_grid = { 
    'n_neighbors': [100,200,300,400],
    'max_depth': [1,2,3,4],
    'criterion': ['gini', 'entropy'],
    'max_features': [0.1, 0.2, 0.25],
}

In [ ]:
CV_rfc = GridSearchCV(knn_classifier, param_grid, cv=3,n_jobs=-1, verbose = 100)
CV_rfc.fit(x_train, y_train)

In [ ]:
print(CV_rfc.cv_results)


#Identify the best score during fitting with cross-validation
print(CV_rfc.best_score_)

In [ ]:
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_test = sc_X.transform(x_test)

In [ ]:
def handle_non(df):
    columns = df.columns.values
        
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]
            
        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x += 1
            df[column] = list(map(convert_to_int, df[column]))
    return df

In [ ]:
df = handle_non(df)

In [ ]:
#df.to_csv('savept3clcars.csv')

In [ ]:
df.head()